**Preprocesamiento de los Datos**

1. Lectura de archivos. Limpiar los registros, guardar en un nuevo archivo .csv
* Estacion Aotizhongxin


In [2]:
#library(dplyr)
#library(tidyr)
data <- read.csv("/content/drive/MyDrive/datos/PRSA_Data_Aotizhongxin_20130301-20170228.csv", header = TRUE, stringsAsFactors = FALSE)

datos_limpios<- data[complete.cases(data),]
head(datos_limpios,5)

# Combinar las columnas "year", "month" y "day" en una columna de fecha
datos_limpios <- datos_limpios %>%
  unite(fecha, year, month, day, sep = "-")


# Guardar los datos actualizados en otro archivo CSV
write.csv(datos_limpios, file = "/content/drive/MyDrive/datos/registros_Aotizhongxin.csv", row.names = FALSE)

# Confirmar que se ha guardado correctamente
cat("Los datos se han guardado en 'archivo_actualizado1.csv'")

 #datos_fecha <- read.csv("C:/csv/archivo_actualizado1.csv")

,No,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,wd,WSPM,station
,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>
1,1,2013,3,1,0,4,4,4,7,300,77,-0.7,1023.0,-18.8,0,NNW,4.4,Aotizhongxin
2,2,2013,3,1,1,8,8,4,7,300,77,-1.1,1023.2,-18.2,0,N,4.7,Aotizhongxin
3,3,2013,3,1,2,7,7,5,10,300,73,-1.1,1023.5,-18.2,0,NNW,5.6,Aotizhongxin
4,4,2013,3,1,3,6,6,11,11,300,72,-1.4,1024.5,-19.4,0,NW,3.1,Aotizhongxin
5,5,2013,3,1,4,3,3,12,12,300,72,-2.0,1025.2,-19.5,0,N,2.0,Aotizhongxin


Los datos se han guardado en 'archivo_actualizado1.csv'

2. Calculos con las variables independientes, calculo de la variable de valor.
* Estacion Aotizhongxin
* Para los gases S02,NO2, PM10, se calcula la media aritmtica.
* Para los gases c0 y 03 se calcula el valor maximo.
* Calculo del AQI



In [8]:
datos_fecha <- read.csv("/content/drive/MyDrive/datos/registros_Aotizhongxin.csv")

# Calcular la media de SO2, NO2 y PM10 agrupados por día
mean_so2 <- datos_fecha %>%   group_by(station, fecha) %>%   summarise(mean_so2 = mean(SO2), .groups= 'drop')

mean_no2 <- datos_fecha %>%   group_by(station, fecha) %>%   summarise(mean_no2 = mean(NO2), .groups= 'drop')

mean_pm10 <- datos_fecha %>%   group_by(station, fecha) %>%   summarise(mean_pm10 = mean(PM10), .groups= 'drop')

# Calcular el valor maximo de C0, o3
max_CO <- datos_fecha %>% group_by(station, fecha) %>% summarise(CO = max(CO), .groups= 'drop')
max_O3 <- datos_fecha %>% group_by(station, fecha) %>% summarise(O3 = max(O3), .groups= 'drop')

#...................................................

calculate_AQI_component <- function(concentration, pollutant) {
   if (pollutant == "PM10") {
      breakpoints <- c(0, 54, 154, 254, 354, 424, 504, 604)
   } else if (pollutant == "SO2") {
      breakpoints <- c(0, 91.7, 196.5, 484.7, 796.48, 1582.48, 2630.48)
   } else if (pollutant == "NO2") {
      breakpoints <- c(0, 99.64, 188, 676.8, 1220.12, 2348.12, 3852.12)
   } else if (pollutant == "CO") {
      breakpoints <- c(0, 5060, 10810, 14260, 17710, 34960, 57960)
   } else if (pollutant == "O3") {
      breakpoints <- c(0, 105.84, 137.2, 166.6, 205.8, 392, 604)
   } else {
      return(rep("Invalid pollutant", length(concentration)))
   }

   index <- c(0, 50, 100, 150, 200, 300, 400, 500)

   aqi <- rep(NA, length(concentration))
   for (i in 1:(length(breakpoints) - 1)) {
      mask <- concentration >= breakpoints[i] & concentration <= breakpoints[i + 1]
      aqi[mask] <- ((index[i + 1] - index[i]) / (breakpoints[i + 1] - breakpoints[i])) * (concentration[mask] - breakpoints[i]) + index[i]
   }

   aqi[is.na(aqi)] <- "Out of range"
   return(round(as.numeric(aqi)))
}

#......................Calculo el evalor del AQI.............................

aqi_so2 <- calculate_AQI_component(mean_so2$mean_so2, "SO2")
aqi_no2 <- calculate_AQI_component(mean_no2$mean_no2, "NO2")
aqi_pm10 <- calculate_AQI_component(mean_pm10$mean_pm10, "PM10")
aqi_CO <- calculate_AQI_component(max_CO$CO, "CO")
aqi_O3 <- calculate_AQI_component(max_O3$O3, "O3")


# Crear un data frame para almacenar los resultados del AQI
aqi_data <- data.frame( fecha = mean_so2$fecha, AQI_SO2 = aqi_so2, AQI_NO2 = aqi_no2, AQI_PM10 = aqi_pm10, AQI_CO = aqi_CO, AQI_O3 = aqi_O3 , ciudad=mean_no2$station  )

# Imprimir el data frame con los resultados del AQI
#print(aqi_data)




df_datos <- aqi_data %>%
  rowwise() %>%
  mutate(AQI = max(c( AQI_PM10,AQI_SO2, AQI_NO2,AQI_CO,AQI_O3)))
 # ungroup()

#write.csv(df_datos, file = "C:/Proyecto AQI/datos/calidad_aire3.csv", row.names = FALSE)



niveles <- c(0, 50, 100, 150, 200, 250, 300, 600)
labels <- c('Excelent', 'Good', 'Slightly Polluted', 'Lightly Polluted', 'Moderately Polluted', 'Heavily Polluted', 'Severely Polluted')

aqi_data$Nivel_Aire <- cut(df_datos$AQI,breaks = niveles, labels = labels)

nuevo_df <- df_datos %>%
  mutate(Nivel_Aire = cut(AQI, breaks = niveles, labels = labels))

write.csv(nuevo_df, file = "/content/drive/MyDrive/datos/AQI_Aotizhongxin.csv", row.names = FALSE)
head(nuevo_df,10)


fecha,AQI_SO2,AQI_NO2,AQI_PM10,AQI_CO,AQI_O3,ciudad,AQI,Nivel_Aire
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<fct>
2013-10-1,7,36,85,22,21,Aotizhongxin,85,Good
2013-10-10,26,27,92,17,21,Aotizhongxin,92,Good
2013-10-11,18,37,72,27,43,Aotizhongxin,72,Good
2013-10-12,3,41,66,13,37,Aotizhongxin,66,Good
2013-10-13,4,32,51,14,25,Aotizhongxin,51,Good
2013-10-14,2,24,35,11,17,Aotizhongxin,35,Excelent
2013-10-15,3,25,24,8,25,Aotizhongxin,25,Excelent
2013-10-16,4,34,64,13,29,Aotizhongxin,64,Good
2013-10-17,12,48,108,18,26,Aotizhongxin,108,Slightly Polluted


3. Archivo consolidado de todas las estaciones a un solo archivo
* Lectura de archivos. Limpiar los registros, guardar en un nuevo archivo.


In [10]:
ruta_archivo1 <- "/content/drive/MyDrive/datos/PRSA_Data_Aotizhongxin_20130301-20170228.csv"
ruta_archivo2 <- "/content/drive/MyDrive/datos/PRSA_Data_Changping_20130301-20170228.csv"
ruta_archivo3 <- "/content/drive/MyDrive/datos/PRSA_Data_Dingling_20130301-20170228.csv"
ruta_archivo4 <- "/content/drive/MyDrive/datos/PRSA_Data_Dongsi_20130301-20170228.csv"
ruta_archivo5 <- "/content/drive/MyDrive/datos/PRSA_Data_Guanyuan_20130301-20170228.csv"
ruta_archivo6 <- "/content/drive/MyDrive/datos/PRSA_Data_Gucheng_20130301-20170228.csv"
ruta_archivo7 <- "/content/drive/MyDrive/datos/PRSA_Data_Huairou_20130301-20170228.csv"
ruta_archivo8 <- "/content/drive/MyDrive/datos/PRSA_Data_Nongzhanguan_20130301-20170228.csv"
ruta_archivo9 <- "/content/drive/MyDrive/datos/PRSA_Data_Shunyi_20130301-20170228.csv"
ruta_archivo10 <- "/content/drive/MyDrive/datos/PRSA_Data_Tiantan_20130301-20170228.csv"
ruta_archivo11 <- "/content/drive/MyDrive/datos/PRSA_Data_Wanliu_20130301-20170228.csv"
ruta_archivo12 <- "/content/drive/MyDrive/datos/PRSA_Data_Wanshouxigong_20130301-20170228.csv"

library(dplyr)
data_aire <- bind_rows(
read.csv(ruta_archivo1),
read.csv(ruta_archivo2),
read.csv(ruta_archivo3),
read.csv(ruta_archivo4),
read.csv(ruta_archivo5),
read.csv(ruta_archivo6),
read.csv(ruta_archivo7),
read.csv(ruta_archivo8),
read.csv(ruta_archivo9),
read.csv(ruta_archivo10),
read.csv(ruta_archivo11),
read.csv(ruta_archivo12)
)

datos_consolidados <- data_aire[complete.cases(data_aire),]
#library(dplyr)
library(tidyr)

datos_consolidados <- datos_consolidados %>%
  unite(fecha, year, month, day, sep = "-")

write.csv(datos_consolidados, file = "/content/drive/MyDrive/datos/archivo_consolidado.csv", row.names = FALSE)

cat("Los datos se han guardado en '/content/drive/MyDrive/datos/archivo_consolidado.csv'")

 #datos_fecha <- read.csv("C:/csv/archivo_actualizado1.csv")

Los datos se han guardado en '/content/drive/MyDrive/datos/archivo_consolidado.csv'

4. Calculo del AQI, todas las estaciones.
* Para los gases S02,NO2, PM10, se calcula la media aritmtica.
* Para los gases c0 y 03 se calcula el valor maximo.
* Calculo del AQI

In [11]:
datos_fecha <- read.csv("/content/drive/MyDrive/datos/archivo_consolidado.csv")

# Calcular la media de SO2, NO2 y PM10 agrupados por día
mean_so2 <- datos_fecha %>%   group_by(station, fecha) %>%   summarise(mean_so2 = mean(SO2), .groups= 'drop')

mean_no2 <- datos_fecha %>%   group_by(station, fecha) %>%   summarise(mean_no2 = mean(NO2), .groups= 'drop')

mean_pm10 <- datos_fecha %>%   group_by(station, fecha) %>%   summarise(mean_pm10 = mean(PM10), .groups= 'drop')

# Calcular el valor maximo de C0, o3
max_CO <- datos_fecha %>% group_by(station, fecha) %>% summarise(CO = max(CO), .groups= 'drop')
max_O3 <- datos_fecha %>% group_by(station, fecha) %>% summarise(O3 = max(O3), .groups= 'drop')

#...................................................

calculate_AQI_component <- function(concentration, pollutant) {
   if (pollutant == "PM10") {
      breakpoints <- c(0, 54, 154, 254, 354, 424, 504, 604)
   } else if (pollutant == "SO2") {
      breakpoints <- c(0, 91.7, 196.5, 484.7, 796.48, 1582.48, 2630.48)
   } else if (pollutant == "NO2") {
      breakpoints <- c(0, 99.64, 188, 676.8, 1220.12, 2348.12, 3852.12)
   } else if (pollutant == "CO") {
      breakpoints <- c(0, 5060, 10810, 14260, 17710, 34960, 57960)
   } else if (pollutant == "O3") {
      breakpoints <- c(0, 105.84, 137.2, 166.6, 205.8, 392, 604)
   } else {
      return(rep("Invalid pollutant", length(concentration)))
   }

   index <- c(0, 50, 100, 150, 200, 300, 400, 500)

   aqi <- rep(NA, length(concentration))
   for (i in 1:(length(breakpoints) - 1)) {
      mask <- concentration >= breakpoints[i] & concentration <= breakpoints[i + 1]
      aqi[mask] <- ((index[i + 1] - index[i]) / (breakpoints[i + 1] - breakpoints[i])) * (concentration[mask] - breakpoints[i]) + index[i]
   }

   aqi[is.na(aqi)] <- "Out of range"
   return(round(as.numeric(aqi)))
}

#......................Calculo el evalor del AQI.............................

aqi_so2 <- calculate_AQI_component(mean_so2$mean_so2, "SO2")
aqi_no2 <- calculate_AQI_component(mean_no2$mean_no2, "NO2")
aqi_pm10 <- calculate_AQI_component(mean_pm10$mean_pm10, "PM10")
aqi_CO <- calculate_AQI_component(max_CO$CO, "CO")
aqi_O3 <- calculate_AQI_component(max_O3$O3, "O3")


# Crear un data frame para almacenar los resultados del AQI
aqi_data <- data.frame( fecha = mean_so2$fecha, AQI_SO2 = aqi_so2, AQI_NO2 = aqi_no2, AQI_PM10 = aqi_pm10, AQI_CO = aqi_CO, AQI_O3 = aqi_O3 , ciudad=mean_no2$station  )

# Imprimir el data frame con los resultados del AQI
#print(aqi_data)




df_datos <- aqi_data %>%
  rowwise() %>%
  mutate(AQI = max(c( AQI_PM10,AQI_SO2, AQI_NO2,AQI_CO,AQI_O3)))
 # ungroup()

#write.csv(df_datos, file = "C:/Proyecto AQI/datos/calidad_aire3.csv", row.names = FALSE)



niveles <- c(0, 50, 100, 150, 200, 250, 300, 600)
labels <- c('Excelent', 'Good', 'Slightly Polluted', 'Lightly Polluted', 'Moderately Polluted', 'Heavily Polluted', 'Severely Polluted')

aqi_data$Nivel_Aire <- cut(df_datos$AQI,breaks = niveles, labels = labels)

nuevo_df <- df_datos %>%
  mutate(Nivel_Aire = cut(AQI, breaks = niveles, labels = labels))

write.csv(nuevo_df, file = "/content/drive/MyDrive/datos/AQI_Estaciones.csv", row.names = FALSE)
head(nuevo_df,10)

Warning message in calculate_AQI_component(max_O3$O3, "O3"):
“NAs introduced by coercion”


fecha,AQI_SO2,AQI_NO2,AQI_PM10,AQI_CO,AQI_O3,ciudad,AQI,Nivel_Aire
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<fct>
2013-10-1,7,36,85,22,21,Aotizhongxin,85,Good
2013-10-10,26,27,92,17,21,Aotizhongxin,92,Good
2013-10-11,18,37,72,27,43,Aotizhongxin,72,Good
2013-10-12,3,41,66,13,37,Aotizhongxin,66,Good
2013-10-13,4,32,51,14,25,Aotizhongxin,51,Good
2013-10-14,2,24,35,11,17,Aotizhongxin,35,Excelent
2013-10-15,3,25,24,8,25,Aotizhongxin,25,Excelent
2013-10-16,4,34,64,13,29,Aotizhongxin,64,Good
2013-10-17,12,48,108,18,26,Aotizhongxin,108,Slightly Polluted
